# Gather Evidence Using PaperQA2

In this notebook, you'll use the PaperQA2 library from FutureHouse to gather evidence from PubMedCentral (PMC) articles.

## 1. Prerequisites

- Python 3.10 or later
- AWS account configured with appropriate permissions
- Access to the Anthropic Claude Sonnet 4 model on Amazon Bedrock
- Basic understanding of Python programming

In [ ]:

%pip install -U boto3 strands-agents strands-agents-tools defusedxml httpx bedrock_agentcore_starter_toolkit paper-qa

## 2. Experiment with PaperQA2

### 2.1. Basic usage

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

In [ ]:
from paperqa import Settings, ask
from paperqa.settings import AgentSettings, ParsingSettings, IndexSettings

LLM = "us.anthropic.claude-sonnet-4-20250514-v1:0"

answer_response = await ask(
    "What is PaperQA2?",
    settings=Settings(
        llm=LLM,
        summary_llm=LLM,
        agent=AgentSettings(
            agent_llm=LLM,
            index=IndexSettings(index_directory="indexes"),
        ),
        embedding="bedrock/amazon.titan-embed-text-v2:0",
        paper_directory="my_papers",
        parsing=ParsingSettings(use_doc_details=False),
    ),
)


In [ ]:
def pretty_print_paperqa_results(answer) -> None:
    """Pretty print the output from the PaperQA2 query."""
    session_output = answer.session

    print("**Question**\n")
    print(session_output.question + "\n")
    print(f"**Answer**\n")
    print(session_output.answer + "\n")
    print("**Evidence**\n")
    for context in session_output.contexts:
        print(f"{context.text.name}:\t{context.context}")
    print("**Token Counts**\n")
    print(f"{'Model':<45} {'Input':<8} {'Output':<8}")
    print("-" * 65)
    for model, values in session_output.token_counts.items():
        print(f"{model:<45} {values[0]:<8} {values[1]:<8}")
    print()
    print("**Estimated Cost**\n")
    print(round(session_output.cost, 3))

In [ ]:
pretty_print_paperqa_results(answer_response)

### 2.2. Cost Optimization with Multiple LLMs

Try different llms. From the docs:

- `llm`: LLM for general use including metadata inference and answer generation. Should be 'best' LLM. Uses include:
    1. Inferring citation information from documents when left unspecified
    1. Extracting title, DOI, and authors from citation information when left unspecified
    1. Optionally injecting pre-answer information
    1. Generating an answer given evidence
    1. Optionally injecting post-answer information 
- `summary_llm`: LLM for creating contextual summaries 
- `agent_llm`: LLM inside the agent making tool selections.

In [ ]:
from paperqa import Settings, ask
from paperqa.settings import AgentSettings, ParsingSettings, IndexSettings

LLM = "us.anthropic.claude-sonnet-4-20250514-v1:0"
SUMMARY_LLM = "bedrock/openai.gpt-oss-120b-1:0"

answer_response = await ask(
    "What is PaperQA2?",
    settings=Settings(
        llm=LLM,
        summary_llm=SUMMARY_LLM,
        agent=AgentSettings(
            agent_llm=LLM,
            index=IndexSettings(index_directory="indexes"),
        ),
        embedding="bedrock/amazon.titan-embed-text-v2:0",
        paper_directory="my_papers",
        parsing=ParsingSettings(use_doc_details=False),
    ),
)

pretty_print_paperqa_results(answer_response)

By using a different model for the evidence summarization step, we were able to reduce the estimated cost by nearly half!

### 2.3. Biomedical Example

Use PMC11231910 as an example. First, download the text from RODA

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import os

s3 = boto3.client("s3", config=Config(signature_version=UNSIGNED))

pmc_id = "PMC9438179"
bucket_name = "pmc-oa-opendata"
object_key = f"oa_comm/txt/all/{pmc_id}.txt"
local_file_path = f"my_papers/{pmc_id}.txt"
s3.download_file(bucket_name, object_key, local_file_path)

In [ ]:
from paperqa import Settings, ask
from paperqa.settings import AgentSettings, ParsingSettings, IndexSettings

LLM = "us.anthropic.claude-sonnet-4-20250514-v1:0"
SUMMARY_LLM = "bedrock/openai.gpt-oss-120b-1:0"

answer_response = await ask(
    "How safe and effective are GLP-1 drugs for long term use?",
    settings=Settings(
        llm=LLM,
        summary_llm=SUMMARY_LLM,
        agent=AgentSettings(
            agent_llm=LLM,
            index=IndexSettings(index_directory="indexes"),
        ),
        embedding="bedrock/amazon.titan-embed-text-v2:0",
        paper_directory="my_papers",
        parsing=ParsingSettings(use_doc_details=False),
    ),
)

pretty_print_paperqa_results(answer_response)

## 4. Create Strands Agent and tool

There's an example of how to add PaperQA to a Strands tool in `gather_evidence.py`. Let's incorporate it into a test agent and call it directly.

In [ ]:
from strands import Agent
from gather_evidence import gather_evidence_tool

MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"
agent = Agent(tools=[gather_evidence_tool], model=MODEL_ID)

agent.tool.gather_evidence_tool(pmcid="PMC9438179", question="How safe and effective are GLP-1 drugs for long term use?")


## 5. Deploy to Amazon Bedrock AgentCore Runtime

## 6. (Optional) Interact with agent using AgentCore Chat

Follow these steps to open an interactive chat session with your new agent.

1. Open a command line terminal in your notebook environment.
2. Navigate to the project root folder (where `pyproject.toml` is located).
3. Run `pip install .` to install the workshop tools including the chat CLI.
4. Run `agentcore-chat` to launch the CLI.
5. Select the `pmc_abstract_agent` by typing its name or index in the terminal and press Enter.
6. Ask your question at the `You:` prompt and press Enter.


## 7. (Optional) Clean Up